## Approach : Chunking the longer text into fix windows size of 512 (127 head tokens + Stop token + 382 tail tokens) and feeding entire 512 tokens in one go

### Step 1: Load relevant packages

In [4]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 0)
import pandas as pd
import spacy
import time 
import nltk
# nltk.download('punkt')
pd.options.display.float_format = '{:.10f}'.format
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import pytextrank
from tqdm.notebook import tqdm
tqdm.pandas()


### Step 2: Load newly trained model

In [5]:
model_checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment"

In [6]:
# # initialize our model and tokenizer
def_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast = True)
def_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
# old_sentiment_pipeline = pipeline("sentiment-analysis", model=old_model ,tokenizer=old_tokenizer,  top_k=5)

### Step 3: Custom functions to increase text readability + head and tail 512 text + compute sentiment 

#### Step 3.1: Cleaning Function

In [7]:
def clean_string(text:str) -> str:
    try: 
        # utf8_apostrophe = b'\xe2\x80\x99'.decode("utf8")
        DATE_REGEX = r"\d+\/\d+\/\d+"
        text = re.sub(r"([^0-9A-Za-z àâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ\~\°\&\“\%\'\:\;\!\-\’\"\.\,\?\€\$\t\n`(\d+\/\d+\/\d+)``\(.*\)`])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
        text = re.sub(' {2,}', ' ', text) # remove inconsistent spaces
        text = re.sub(' \. ', '.', text) # remove inconsistent spaces
        # text = re.sub(utf8_apostrophe, "'", text) #Subsitute \' with ' 
        text = text.strip()  #strip whitespace
        text = re.sub(r"\n", " ", text)
        text = re.sub(r"\'s\b", "s", text)
        return text
    except Exception as e:
        print(f"Error cleaning text: {e}")
        return text

#### Step 3.2: Create custom 512 token paragraph

In [8]:
def _long_text_head_tail(tokens, chunksize):
    # define target chunksize
    total_length = len(tokens['input_ids'][0])
    head_length = 127
    tail_length = 382
    # If total length is less than 512, select first 128 and last 382 tokens
    head_tokens = tokens['input_ids'][0][:head_length]
    tail_tokens = tokens['input_ids'][0][total_length-tail_length:]
    join_token =  torch.tensor([119]) # This serves as connecting between head and tail but aslso easilty distinguishable
    selected_tokens = torch.cat([head_tokens, join_token, tail_tokens], dim=0)


    # add CLS and SEP tokens
    input_ids = torch.cat([
        torch.tensor([101]), selected_tokens, torch.tensor([102])
    ])

    # add attention tokens to attention mask
    attention_mask = torch.cat([
        torch.tensor([1]), torch.ones_like(selected_tokens), torch.tensor([1])
    ])

    # add padding if necessary
    pad_len = chunksize - input_ids.shape[0]
    if pad_len > 0:
        input_ids = torch.cat([input_ids, torch.zeros(pad_len, dtype=torch.long)])
        attention_mask = torch.cat([attention_mask, torch.zeros(pad_len, dtype=torch.long)])

    input_dict = {
        'input_ids': input_ids.unsqueeze(0),
        'attention_mask': attention_mask.unsqueeze(0)
    }
    
    return input_dict


#### Step 3.3: function to compute sentiment for entire 512 token paragraph

In [9]:
def _calculate_sentiment(html_content:str, title_sentences:str, name:str, model, tokenizer ):

    # A quick cleaninig of html_content and title
    html_content = clean_string(text = html_content)  #
    title_sentences = clean_string(text = title_sentences) #
    html_content = re.sub(re.escape(title_sentences), "", html_content) # this code eliminates the duplicated title text in html_content

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    chunk_size = 512

    content_inputs = tokenizer.encode_plus(html_content, add_special_tokens=False, return_tensors='pt').to(device)
    if len(content_inputs['input_ids'][0]) > chunk_size - 2:
        input_dict = _long_text_head_tail(content_inputs, chunksize=chunk_size - 2)
        content_sentences = tokenizer.batch_decode(input_dict['input_ids'], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        with torch.no_grad():
            content_outputs = model(**input_dict)
    else:
        content_sentences = html_content
        content_inputs = tokenizer(content_sentences, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            content_outputs = model(**content_inputs)

    
    # Softmax for probability scaling, i.e. sum of all probabilities = 1    
    content_scores = torch.nn.functional.softmax(content_outputs.logits, dim=-1)
    # Define the scale and shift it, in this case to --> -1, -0.5, 0, 0.5, 1
    weights = torch.linspace(-1, 1, content_scores.shape[1])
    normalised_content_scores = torch.matmul(content_scores, weights).tolist()
    
    # For title content
    title_inputs = tokenizer(title_sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():       
        title_outputs = model(**title_inputs)
    title_scores = torch.nn.functional.softmax(title_outputs.logits, dim=-1)
    normalised_title_scores = torch.matmul(title_scores, weights).tolist()

    mentions = 0
    score = 0.0
    count = 0
    for sentence, sentiment in zip(content_sentences, normalised_content_scores):
        multiplier = 1
        if name.lower() in sentence.lower():
            mentions += sentence.lower().count(name.lower())
            multiplier = 5

        count += multiplier
        score += sentiment * multiplier

    for sentence, sentiment in zip(title_sentences, normalised_title_scores):
        mentions += sentence.lower().count(name.lower())
        multiplier = 5
        count += multiplier
        score += sentiment * multiplier

    average_score = score / count
    article_score = average_score * 50 + 50

    return round(article_score, 5)

In [10]:
html_text = "What is Volkswagen accused of?\nIt's been dubbed the \"diesel dupe\". In September, the Environmental Protection Agency (EPA) found that many VW cars being sold in America had a \"defeat device\" - or software - in diesel engines that could detect when they were being tested, changing the performance accordingly to improve results. The German car giant has since admitted cheating emissions tests in the US.\nVW has had a major push to sell diesel cars in the US, backed by a huge marketing campaign trumpeting its cars' low emissions. The EPA's findings cover 482,000 cars in the US only, including the VW-manufactured Audi A3, and the VW models Jetta, Beetle, Golf and Passat. But VW has admitted that about 11 million cars worldwide, including eight million in Europe, are fitted with the so-called \"defeat device\".\nThe company has also been accused by the EPA of modifying software on the 3 litre diesel engines fitted to some Porsche and Audi as well as VW models. VW has denied the claims, which affect at least 10,000 vehicles.\nIn November, VW said it had found \"irregularities\" in tests to measure carbon dioxide emissions levels that could affect about 800,000 cars in Europe - including petrol vehicles. However, in December it said that following investigations, it had established that this only affected about 36,000 of the cars it produces each year.\nThis 'defeat device' sounds like a sophisticated piece of kit.\nFull details of how it worked are sketchy, although the EPA has said that the engines had computer software that could sense test scenarios by monitoring speed, engine operation, air pressure and even the position of the steering wheel.\nWhen the cars were operating under controlled laboratory conditions - which typically involve putting them on a stationary test rig - the device appears to have put the vehicle into a sort of safety mode in which the engine ran below normal power and performance. Once on the road, the engines switched out of this test mode.\nThe result? The engines emitted nitrogen oxide pollutants up to 40 times above what is allowed in the US.\nWhat has been VW's response?\n\"We've totally screwed up,\" said VW America boss Michael Horn, while the group's chief executive at the time, Martin Winterkorn, said his company had \"broken the trust of our customers and the public\". Mr Winterkorn resigned as a direct result of the scandal and was replaced by Matthias Mueller, the former boss of Porsche.\n\"My most urgent task is to win back trust for the Volkswagen Group - by leaving no stone unturned,\" Mr Mueller said on taking up his new post.\nVW has also launched an internal inquiry.\nBut that's unlikely to be the end of the financial impact. The EPA has the power to fine a company up to $37,500 for each vehicle that breaches standards - a maximum fine of about $18bn.\nThe costs of possible legal action by car owners and shareholders \"cannot be estimated at the current time\", VW added.\nHow widespread are VW's problems?\nWhat started in the US has spread to a growing number of countries. The UK, Italy, France, South Korea, Canada and, of course, Germany, have opened investigations. Throughout the world, politicians, regulators and environmental groups are questioning the legitimacy of VW's emissions testing.\nVW will recall 8.5 million cars in Europe, including 2.4 million in Germany and 1.2 million in the UK, and 500,000 in the US as a result of the emissions scandal.\nNo wonder the carmaker's shares have fallen by about a third since the scandal broke.\nWill more heads roll?\nIt's still unclear who knew what and when, although VW must have had a chain of management command that approved fitting cheating devices to its engines, so further departures are likely.\nChristian Klingler, a management board member and head of sales and marketing is leaving the company, although VW said this was part of long-term planned structural changes and was not related to recent events.\nIn 2014, in the US, regulators raised concerns about VW emissions levels, but these were dismissed by the company as \"technical issues\" and \"unexpected\" real-world conditions. If executives and managers wilfully misled officials (or their own VW superiors) it's difficult to see them surviving.\nAre other carmakers implicated?\nThat's for the various regulatory and government inquiries to determine. California's Air Resources Board is now looking into other manufacturers' testing results. Ford, BMW and Renault-Nissan have said they did not use \"defeat devices\", while other firms have either not commented or simply stated that they comply with the law.\nThe UK trade body for the car industry, the SMMT, said: \"The EU operates a fundamentally different system to the US - with all European tests performed in strict conditions as required by EU law and witnessed by a government-appointed independent approval agency.\"\nBut it added: \"The industry acknowledges that the current test method is outdated and is seeking agreement from the European Commission for a new emissions test that embraces new testing technologies and is more representative of on-road conditions.\"\nThat sounds like EU testing rules need tightening, too.\nEnvironmental campaigners have long argued that emissions rules are being flouted. \"Diesel cars in Europe operate with worse technology on average than the US,\" said Jos Dings, from the pressure group Transport & Environment. \"Our latest report demonstrated that almost 90% of diesel vehicles didn't meet emission limits when they drive on the road. We are talking millions of vehicles.\"\nCar analysts at the financial research firm Bernstein agree that European standards are not as strict as those in the US. However, the analysts said in a report that there was, therefore, \"less need to cheat\". So, if other European carmakers' results are suspect, Bernstein says the \"consequences are likely to be a change in the test cycle rather than legal action and fines\".\nIt's all another blow for the diesel market.\nCertainly is. Over the past decade and more, carmakers have poured a fortune into the production of diesel vehicles - with the support of many governments - believing that they are better for the environment. Latest scientific evidence suggests that's not the case, and there are even moves to limit diesel cars in some cities.\nDiesel sales were already slowing, so the VW scandal came at a bad time. \"The revelations are likely to lead to a sharp fall in demand for diesel engine cars,\" said Richard Gane, automotive expert at consultants Vendigital.\n\"In the US, the diesel car market currently represents around 1% of all new car sales and this is unlikely to increase in the short to medium term.\n\"However, in Europe the impact could be much more significant, leading to a large tranche of the market switching to petrol engine cars virtually overnight.\""
summary = "In September, the Environmental Protection Agency (EPA) found that many VW cars being sold in America had a \"defeat device\" - or software - in diesel engines that could detect when they were being tested, changing the performance accordingly to improve results. VW has had a major push to sell diesel cars in the US, backed by a huge marketing campaign trumpeting its cars' low emissions. The company has also been accused by the EPA of modifying software on the 3 litre diesel engines fitted to some Porsche and Audi as well as VW models. In November, VW said it had found \"irregularities\" in tests to measure carbon dioxide emissions levels that could affect about 800,000 cars in Europe - including petrol vehicles. VW will recall 8.5 million cars in Europe, including 2.4 million in Germany and 1.2 million in the UK, and 500,000 in the US as a result of the emissions scandal."
title = "Volkswagen: The scandal explained - BBC News"
name = "Volkswagen"

In [14]:
article_score = _calculate_sentiment(html_content = html_text, title_sentences = title, name=name, model= def_model, tokenizer=def_tokenizer)

In [15]:
article_score

27.28324